In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SalesDataBI") \
    .master("local[1]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

# Loading the Data

## About Dataset from kaggle

Data holds the basic information about sales data. The company have sales agencies / resellers and branches and the data file holds only the branch/reseller information in the customer field.

Notes:
Please note that, for the privacy issues, the customerID, SKU and documentID is processed with LabelEncoder. So, each customer or each product has a unique ID in the data file.

Note 2: Data holds for more than 3 years, prices of the products can get raised by the time.

DocumentID : ID of the transaction. A transaction might hold multiple records for the same customer at the same date with multiple products (SKU). DocumentID might be useful for combining the transactions and detecting the items sold together.
Date : Date of transaction / sell. In the date time format.
SKU : Item / Product code. The unique code for each item sold.
Price : Sales price for the transaction. The price for the product for the customer for the date.
Discount : discount amount for the transaction.
Customer : Unique customer id for each customer. For the data set, customer can be a reseller or a branch of the company.
Quantity : Number of items sold in the transaction.
Version 2 Updates:
Newer version of the data set has more rows until 2023 January. Also some of the column names are updated for the naming standards in other Kaggle notebooks.

InvoiceID : ID of the transaction. A transaction might hold multiple records for the same customer at the same date with multiple products (SKU). DocumentID might be useful for combining the transactions and detecting the items sold together.
Date : Date of transaction / sell. In the date time format.
ProductID : Item / Product code. The unique code for each item sold.
TotalSales : Sales price for the transaction. If you want to get unit_price , divide TotalSales column to Quantity column
Discount : Discount amount for the transaction.
CustomerID : Unique customer id for each customer. For the data set, customer can be a reseller or a branch of the company.
Quantity : Number of items sold in the transaction.

In [10]:
sales_df1 = spark.read.option("header", "true").csv("dataset/sales1.csv")

In [11]:
sales_df1.show(5)

+---+----------+----------+----+------+--------+--------+--------+
|_c0|DocumentID|      Date| SKU| Price|Discount|Customer|Quantity|
+---+----------+----------+----+------+--------+--------+--------+
|  0|       716|2019-09-23|1039|381.78|67.37254|       1|     1.0|
|  1|       716|2019-09-23| 853|593.22| 0.00034|       1|     1.0|
|  2|       716|2019-09-23| 862|423.73|-0.00119|       1|     1.0|
|  3|       716|2019-09-23| 868| 201.7|35.58814|       1|     1.0|
|  4|       716|2019-09-23|2313|345.76|61.01966|       1|     1.0|
+---+----------+----------+----+------+--------+--------+--------+
only showing top 5 rows



24/11/06 03:58:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , DocumentID, Date, SKU, Price, Discount, Customer, Quantity
 Schema: _c0, DocumentID, Date, SKU, Price, Discount, Customer, Quantity
Expected: _c0 but found: 
CSV file: file:///home/ayoub/BI2Bigdata/dataset/sales1.csv


In [12]:
sales_df1 = sales_df1.withColumnRenamed("DocumentID", "InvoiceID") \
                   .withColumnRenamed("SKU", "ProductID") \
                   .withColumnRenamed("Price", "TotalSales") \
                   .withColumnRenamed("Customer", "CustomerID")

In [13]:
sales_df1.show(5)

+---+---------+----------+---------+----------+--------+----------+--------+
|_c0|InvoiceID|      Date|ProductID|TotalSales|Discount|CustomerID|Quantity|
+---+---------+----------+---------+----------+--------+----------+--------+
|  0|      716|2019-09-23|     1039|    381.78|67.37254|         1|     1.0|
|  1|      716|2019-09-23|      853|    593.22| 0.00034|         1|     1.0|
|  2|      716|2019-09-23|      862|    423.73|-0.00119|         1|     1.0|
|  3|      716|2019-09-23|      868|     201.7|35.58814|         1|     1.0|
|  4|      716|2019-09-23|     2313|    345.76|61.01966|         1|     1.0|
+---+---------+----------+---------+----------+--------+----------+--------+
only showing top 5 rows



24/11/06 03:58:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , DocumentID, Date, SKU, Price, Discount, Customer, Quantity
 Schema: _c0, DocumentID, Date, SKU, Price, Discount, Customer, Quantity
Expected: _c0 but found: 
CSV file: file:///home/ayoub/BI2Bigdata/dataset/sales1.csv


In [14]:
sales_df2 = spark.read.option("header", "true").csv("dataset/sales2.csv")

In [15]:
sales_df = sales_df1.union(sales_df2)
sales_df.show(5)

+---+---------+----------+---------+----------+--------+----------+--------+
|_c0|InvoiceID|      Date|ProductID|TotalSales|Discount|CustomerID|Quantity|
+---+---------+----------+---------+----------+--------+----------+--------+
|  0|      716|2019-09-23|     1039|    381.78|67.37254|         1|     1.0|
|  1|      716|2019-09-23|      853|    593.22| 0.00034|         1|     1.0|
|  2|      716|2019-09-23|      862|    423.73|-0.00119|         1|     1.0|
|  3|      716|2019-09-23|      868|     201.7|35.58814|         1|     1.0|
|  4|      716|2019-09-23|     2313|    345.76|61.01966|         1|     1.0|
+---+---------+----------+---------+----------+--------+----------+--------+
only showing top 5 rows



24/11/06 03:59:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , DocumentID, Date, SKU, Price, Discount, Customer, Quantity
 Schema: _c0, DocumentID, Date, SKU, Price, Discount, Customer, Quantity
Expected: _c0 but found: 
CSV file: file:///home/ayoub/BI2Bigdata/dataset/sales1.csv


In [17]:
sales_df = sales_df.drop("_c0")

In [18]:
sales_df.write.format("delta").mode("overwrite").save("delta/sales_data")

# Slowly Changing Dimensions (SCD)

## Customer Dimension Table (SCD Type 2)

In [20]:
customer_data = sales_df.select("CustomerID", "CustomerID").distinct()

In [ ]:
from pyspark.sql import functions as F

customer_dim = DeltaTable.forPath(spark, "/path_to_delta/customer_dimension")


new_customer_data = spark.read.option("header", "true").csv("path_to_new_customer_data.csv")

# Apply SCD Type 2: Update existing records with new customer data
customer_dim.alias("old") \
    .merge(new_customer_data.alias("new"), "old.CustomerID = new.CustomerID") \
    .whenMatchedUpdate(
        condition="old.Customer != new.Customer",
        set={"valid_to": F.current_date(), "is_active": F.lit(False)}  # Expire old record
    ) \
    .whenNotMatchedInsert(
        values={
            "CustomerID": "new.CustomerID",
            "Customer": "new.Customer",
            "valid_from": F.current_date(),
            "is_active": F.lit(True)
        }
    ) \
    .execute()